# Step-by-step implementation
The following are the steps to implement multi-query:
  1. Import necessary modules
  2. Set up the LangSmith and OpenAI API keys
  3. Prepare data and split text
  4. Index documents
  5. Generate multi-perspective query with LLM
  6. Retrieve documents using multi-query
  7. Run the RAG model


## 1. Import necessary modules

In [0]:
import os
import bs4
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

## 2. Set up the LangSmith and OpenAI API keys

In [0]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '' # Add your LangSmith LangChain API key
os.environ['LANGCHAIN_PROJECT']='Multi-Query'

In [0]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"] = ""  # Add your OpenAI API key
if OPENAI_API_KEY == "":
    raise ValueError("Please set the OPENAI_API_KEY environment variable")

## 3. Prepare data and split text

In [0]:
loaders = [
    TextLoader("blog.langchain.dev_announcing-langsmith_.txt"),
    TextLoader("blog.langchain.dev_automating-web-research_.txt"),
]

docs = []
for loader in loaders:
    docs.extend(loader.load())

In [0]:
docs

In [0]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=400, chunk_overlap=60)
splits = text_splitter.split_documents(docs)

## 4. Index documents

In [0]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

## 5. Generate multi-perspective query with LLM

In [0]:
# Multi Query: Different Perspectives
template = """You are an AI language model assistant tasked with generating informative queries for a vector search engine.
The user has a question: "{question}"
Your goal/task is to create three variations of this question that capture different aspects of the user's intent. These variations will help the search engine retrieve relevant documents even if they don't use the exact keywords as the original question.
Provide these alternative questions, each on a new line.**
Original question: {question}"""

prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspectives
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

## 6. Retrieve documents using multi-query

In [0]:
def get_unique_union(documents: list[list]):
  """ Unique union of retrieved docs """
  # Flatten list of lists
  flattened_docs = [doc for sublist in documents for doc in sublist]

  # Option 1: Check library documentation for hashable attribute (e.g., 'id')
  if hasattr(flattened_docs[0], 'id'):  # Replace 'id' with the appropriate attribute
      unique_docs = list(set(doc.id for doc in flattened_docs))

  # Option 2: Convert to string (if suitable)
  else:
      unique_docs = list(set(str(doc) for doc in flattened_docs))

  return unique_docs

In [0]:
# Retrieve
question = "What is LangSmith, and why do we need it?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

## 7. Run the RAG model

In [0]:
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature=0)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})